<a href="https://colab.research.google.com/github/ElzatAhmed/machine-learning-notes/blob/main/gnn_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy
!pip install spektral

import numpy as np
import tensorflow as tf
import spektral

In [17]:
cora = spektral.datasets.citation.Citation(name='cora')
graph = cora.read()
train_mask, val_mask, test_mask = cora.mask_tr, cora.mask_va, cora.mask_te
adj, features, labels = graph[0].a, graph[0].x, graph[0].y

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [21]:
print(features.shape)
print(adj.shape)
print(labels.shape)
print(np.sum(train_mask))
print(np.sum(val_mask))
print(np.sum(test_mask))

(2708, 1433)
(2708, 2708)
(2708, 7)
140
500
1000


In [32]:
print(labels[100])

[1. 0. 0. 0. 0. 0. 0.]


In [ ]:
def masked_softmax_cross_entropy(logits, labels, masks):
  loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)
  mask = tf.cast(tf.float32)
  mask /= tf.reduce_mean(mask)
  loss *= mask
  return tf.reduce_mean(loss)


def masked_accuracy(logits, labels, masks):
  correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
  accuracy_all = tf.cast(correct_prediction, tf.float32)
  mask = tf.cast(tf.float32)
  mask /= tf.reduce_mean(mask)
  accuracy_all *= mask
  return tf.reduce_mean(accuracy_all)

In [ ]:
def gnn(features, adj, transform, activation):
  seq_fts = transform(features)
  ret_fts = tf.matmul(adj, seq_fts)
  return activation(ret_fts)

In [ ]:
def train_cora(features, adj, gnn_fn, units, epochs, learning_rate):
  layer1 = tf.keras.layers.Dense(units=units)
  layer2 = tf.keras.layers.Dense(units=7)

  def cora_gnn(features, adj):
    hidden = gnn_fn(features, adj, layer1, tf.nn.relu)
    logits = gnn_fn(features, adj, layer2, tf.nn.identity)
    return logits
  
  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

  # TODO: complete the neural network